In [313]:
import numpy as np
import csv

import sys
sys.path.append('../../')
import pyGM as gm;
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
with open("../league of legends/cut_data/lol_cut.csv", newline='') as i:
    reader = csv.reader(i)
    lol_data= list(reader)

In [314]:
for i in range(5):
    for j in lol_data[i]:
        print(j)

gameId
creationTime
gameDuration
seasonId
winner
firstBlood
firstTower
firstInhibitor
firstBaron
firstDragon
firstRiftHerald
3326086514
1504279457970
1949
9
1
2
1
1
1
1
2
3229566029
1497848803862
1851
9
1
1
1
1
0
1
1
3327363504
1504360103310
1493
9
1
2
1
1
1
2
0
3326856598
1504348503996
1758
9
1
1
1
1
1
1
0


# turn the data into something managable by pygm

In [315]:
#using 6 combinations, one for each first objective


first_combinations=2
nlevels=10
scale=.2

X = [None]*first_combinations
for i in range(first_combinations):
    X[i] = gm.Var(i, nlevels)  

del lol_data[0] #delete the header to avoid fuck ups
Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j                   # find the advantage of Pi over Pj, then 
        Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage
print(Pwin)


factors = [ gm.Factor([X[i]],1./nlevels) for i in range(first_combinations) ] #uniform belief over team skill
    

[[0.5        0.450166   0.40131234 0.35434369 0.31002552 0.26894142
  0.23147522 0.19781611 0.16798161 0.14185106]
 [0.549834   0.5        0.450166   0.40131234 0.35434369 0.31002552
  0.26894142 0.23147522 0.19781611 0.16798161]
 [0.59868766 0.549834   0.5        0.450166   0.40131234 0.35434369
  0.31002552 0.26894142 0.23147522 0.19781611]
 [0.64565631 0.59868766 0.549834   0.5        0.450166   0.40131234
  0.35434369 0.31002552 0.26894142 0.23147522]
 [0.68997448 0.64565631 0.59868766 0.549834   0.5        0.450166
  0.40131234 0.35434369 0.31002552 0.26894142]
 [0.73105858 0.68997448 0.64565631 0.59868766 0.549834   0.5
  0.450166   0.40131234 0.35434369 0.31002552]
 [0.76852478 0.73105858 0.68997448 0.64565631 0.59868766 0.549834
  0.5        0.450166   0.40131234 0.35434369]
 [0.80218389 0.76852478 0.73105858 0.68997448 0.64565631 0.59868766
  0.549834   0.5        0.450166   0.40131234]
 [0.83201839 0.80218389 0.76852478 0.73105858 0.68997448 0.64565631
  0.59868766 0.549834  

In [316]:
for i in factors:
    print(i)

Factor({0})
Factor({1})


In [317]:
#turn the csv into str because oh my god csv's are all strings thats so stupid
for i in range(len(lol_data)):
    for j in range(len(lol_data[i])):
        lol_data[i][j]=int(lol_data[i][j])


In [318]:
#adding game info
print(len(X))
print(len(factors))
for g in lol_data:
    blood,turret,inhib,baron,dragon,harold,win = g[5],g[6],g[7],g[8],g[9],g[10],g[4]
    temp=[blood,turret,inhib,baron,dragon,harold,win]
    
    #check who got the most amount of firsts
    t1=0
    t2=0
    for i in temp:
        if i ==1:
            t1+=1
        elif i==2:
            t2+=1
            
    
    #if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
    
    #for every combination of values, there are two teams and one won.
    #each X represents wether or not they got a first objective.
    #it will always be in the order thats in the csv (fb=5,ft=6,fi=7,fbaron=8,fd=9,fh=10)
    #factors.append(gm.Factor([X[blood],X[turret]], Pwin if win>0 else 1-Pwin) )
        
    factors.append(gm.Factor( [X[0],X[1]], Pwin if win==1 else 1-Pwin))

2
2


In [319]:
# for i in factors:

#     for j in i.table:
#         if type(j) == np.float64:
#             continue
#         elif 0 in j:
#             print(table)

In [320]:
model = gm.GraphModel(factors)
print("its working")
#model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)

its working


In [321]:
if model.nvar < 0:       # for very small models, we can do brute force inference:
    jt = model.joint()
    jt /= jt.sum()       # normalize the distribution and marginalize the table
    bel = [jt.marginal([i]) for i in range(nplayers)] 
else:                    # otherwise we need to use some approximate inference:
    from pyGM.messagepass import LBP, NMF
    #lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
    lnZ,bel = NMF(model, maxIter=3, verbose=True)  # Mean field

Iter 0: -39692.19621587434
Iter 1: -35694.753497175145
Iter 2: -35694.753497175145
Iter 3: -35694.753497175145


In [326]:
i,j = 0,1
print("Estimated probability P{} beats P{} next time:".format(i,j))
# Expected value (over skill of P0, P1) of Pr[win | P0-P1]
for i in bel:
    print(i.table)
if i<j:
    print( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],Pwin)).table.sum() )
else:
    print( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],1-Pwin)).table.sum() )

Estimated probability P0 beats P1 next time:
[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 3.46405783e-141 1.00000000e+000 1.63641078e-083 0.00000000e+000
 0.00000000e+000 0.00000000e+000]
[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 1.63641078e-083 1.00000000e+000 3.46405783e-141 0.00000000e+000
 0.00000000e+000 0.00000000e+000]


TypeError: '<' not supported between instances of 'Factor' and 'int'